# Data Analytics Project Work
### File for testing different stuff for the project :D

In [68]:
# imports
import pandas as pd
import numpy as np

# Testing on reading one of the datasets and creating a Pandas DataFrame - object
test_df = pd.read_csv("HybridDataset_for_RSSbasedLocalization-main/Scenario 0/RPI/RSS_BLT_Dataset/Distanza10.csv", sep=';')
test_df.head()

,Timestamp,Rx Power [dBm]
0,Fri Mar 24 14:13:33 2023,-79
1,Fri Mar 24 14:13:34 2023,-70
2,Fri Mar 24 14:13:36 2023,-73
3,Fri Mar 24 14:13:38 2023,-79
4,Fri Mar 24 14:13:39 2023,-76


In [69]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Timestamp       517 non-null    object
 1   Rx Power [dBm]  517 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.2+ KB


Everything seemed to be working great as we got the first five entries of the dataset with a .head()– method and some information about the dataset with the .info()– method.

In [70]:
def read_data(anchors, directory_path):
    """
    Reads the csv-files for one specified measurement case and adds them into a dict of dataframes.
    
    Parameters:
    anchors (list): List of anchors, for example ["Anchor 1", "Anchor 2", "Anchor 3"...]
    directory path (string): The relative directory path of the folder which contains the wanted anchor-csv.files.

    returns: 
    dict: a dict which keys are the anchor names and values are the respective dataframes
    """
    scenario_dataframes = {}

    for anchor in anchors:
        current_df = pd.read_csv(directory_path + anchor + ".csv", sep=';')
        scenario_dataframes[anchor.replace(".csv","")] = current_df
    
    return scenario_dataframes


In [71]:
def calculate_mean_values(scenario_dataframe, bursts):
    """
    Calculates the mean value over a given amount of bursts.

    Parameters:
    scenario_dataframe (DataFrame): A DataFrame-object containing the dataset of one anchor.
    bursts (int): The amount of bursts.

    Returns:
    int: Returns the mean value over the given amount of bursts.
    """

    mean_values = []

    power_values = scenario_dataframe["Rx Power [dBm]"].to_list()
    # list_size = len(power_values)
    # burst_size = list_size // bursts

    start = 0
    for i in range(bursts):
        # end = start + burst_size if i < bursts - 1 else list_size - 1
        mean_values.append(power_values[i])
        # start = end + 1
        
    mean_value = np.mean(mean_values)
    
    return mean_value

In [72]:
def calculate_estimates(p_initial, n_value, mean_values):
    """
    Calculates the distance estimates for each anchor.

    Parameteres:
    p_initial (int): The initial RSS value for the distance estimation equation.
    n_value (int): The n_value for the distance estimation formula.
    mean_values (dict): A dict which keys are the anchor names and values are their respective mean values over the given amount of bursts

    Returns:
    dict: Returns a dict which keys are the anchor names and values are their respective distance estimations.
    """
    estimated_distances = {}

    for anchor in mean_values:
        estimated_d = 10**((p_initial-mean_values[anchor])/(10*n_value))
        estimated_distances[anchor] = estimated_d

    return estimated_distances

In [73]:
def get_actual_distance(anchor_dataframe):
    """
    Extracts the first element of the last column ("Distance Target - Anchor [m]") from the DataFrame.

    Parameters:
    anchor_dataframe (DataFrame): A DataFrame containing the dataset of one anchor.

    Returns:
    float: Returns the first element of the last column.
    """
    distance = anchor_dataframe.at[0, "Distance Target - Anchor [m]"]
    return float(distance)



In [74]:
def extract_anchor_coordinates(dataframe):
    # Assuming the coordinates are in the 'Relative Coordinates [m]' column
    first_row = dataframe.iloc[0]
    
    # Extracting the 'Relative Coordinates [m]' from the first row
    relative_coordinates_str = first_row['Relative Coordinates [m]']
    
    # Splitting the string by comma and space, then taking the first two elements
    coordinates_split = pd.to_numeric(relative_coordinates_str.split(', '), errors='raise')

    # coordinates_split is a numpy.ndarray type: n-dimensional array of items
    print("extracted: ", int(coordinates_split))
    return coordinates_split

In [75]:
def calculate_wls_position(A, W, b):
    # Solve the system of equations using the Weighted Least Squares method
    A_wls = np.dot(np.dot(np.linalg.inv(np.dot(np.dot(A.T, W), A)), A.T), W)
    estimated_position = np.dot(A_wls, b)

    return estimated_position

In [76]:
def calculate_wls_matrix(anchors, estimated_distances):
    # Build matrix A and vector b for the WLS method
    A = np.array([[-2 * anchor[0], -2 * anchor[1], 1] for anchor in anchors])
    b = np.array([estimated_distances[i] for i in range(len(anchors))])

    return A, b

Let's estimate the distances of the anchors in Scenario A RPI BLT_dataset

In [77]:

anchors = ["Anchor 1", "Anchor 2", "Anchor 3", "Anchor 4", "Anchor 5", "Anchor 6"]
directory_path = "HybridDataset_for_RSSbasedLocalization-main/Scenario A/RPI/RSS_BLT_Dataset/"

# Bluetooth Scenario A, all anchors
dataframes = read_data(anchors, directory_path)
bursts = 10

mean_values = {}
actual_distances = {}
for anchor in dataframes:
    mean_values[anchor] = calculate_mean_values(dataframes[anchor], bursts)
    actual_distances[anchor] = get_actual_distance(dataframes[anchor])
    print(anchor, "mean value over", bursts, "bursts:", mean_values[anchor])

p_initial = -40
n_value = 4
estimated_distances = calculate_estimates(p_initial, n_value, mean_values)

print("")
for anchor in anchors:
    print(anchor, "Estimated distance", estimated_distances[anchor], " vs Actual distance:", actual_distances[anchor])
    
# Position estimation
# Extract coordinates of anchors from their respective CSV files
anchor_coordinates = {}
for anchor in anchors:
    anchor_dataframe = dataframes[anchor]
    anchor_coordinates[anchor] = extract_anchor_coordinates(anchor_dataframe)

# Print the extracted coordinates
# for anchor, coordinates in anchor_coordinates.items():
#     print(f"{anchor} Coordinates: {coordinates}")

# # Calculate WLS position estimates for each anchor
# wls_positions = {}
# for anchor in anchors:
#     A, b = calculate_wls_matrix([anchor_coordinates[anchor].tolist()], estimated_distances[anchor])
    
#     # Assuming W is a diagonal matrix with variances (inverse of variances)
#     W = np.identity(len(estimated_distances[anchor]))
    
#     estimated_position = calculate_wls_position(A, W, b)
    
#     wls_positions[anchor] = estimated_position

# # Print the WLS position estimates
# print("\nWLS Position Estimates:")
# for anchor, position in wls_positions.items():
#     print(f"{anchor} Estimated Position: {position}")

    

Anchor 1 mean value over 10 bursts: -71.8
Anchor 2 mean value over 10 bursts: -77.1
Anchor 3 mean value over 10 bursts: -67.8
Anchor 4 mean value over 10 bursts: -75.2
Anchor 5 mean value over 10 bursts: -73.4
Anchor 6 mean value over 10 bursts: -59.5

Anchor 1 Estimated distance 6.237348354824191  vs Actual distance: 15.994
Anchor 2 Estimated distance 8.46252568807269  vs Actual distance: 15.994
Anchor 3 Estimated distance 4.954501908047901  vs Actual distance: 10.55
Anchor 4 Estimated distance 7.58577575029184  vs Actual distance: 15.0
Anchor 5 Estimated distance 6.839116472814296  vs Actual distance: 12.944
Anchor 6 Estimated distance 3.0725573652674467  vs Actual distance: 12.944
extracted:  [ 5.55 15.  ]
extracted:  [ 5.55 15.  ]
extracted:  [-10.55   0.  ]
extracted:  [  0 -15]
extracted:  [-10.55  -7.5 ]
extracted:  [-10.55  -7.5 ]
